In [1]:
def peel_first_col(fname, fcol, delimiter=None):
    with open(fname, 'r') as fin:
        for line in fin:
            try:
                fcol.append(line.split(delimiter, 1)[0])
                yield line.split(delimiter, 1)[1]
            except IndexError:
                continue

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import networkx as nx
import string

symMats = []
graphs = []
algos = []
col = []
algoOrder = ["PoAV", "PQAV", "Po2P", "PQ2P", "PoPN", "PQPN"]
ext =       ["_0",   "_1",   "_2",   "_3",   "_4",   "_5"]
for e in range(len(ext)):
    col = []
    klDiv = np.loadtxt(peel_first_col("../output/cmGrouping_bigscanfchr5/ARR"+ext[e]+".mx", col), skiprows=1)
    col.pop(0)
    #klDiv = np.absolute(klDiv)
    symKlDiv = np.triu(klDiv)+np.tril(klDiv).T
    symMats = symMats + [symKlDiv]

    G = nx.from_numpy_matrix(symKlDiv)
    for i in range(len(col)):
        G.node[i]['id'] = col[i]
    graphs = graphs + [G]
    algos = algos + [algoOrder[e]]
print(np.array(col).T)

FileNotFoundError: [Errno 2] No such file or directory: '../output/cmGrouping_bigscanfchr5/ARR_0.mx'

In [ ]:
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn import metrics

affMats = []
for i in range(len(symMats)):
    adj_mat = symMats[i] + symMats[i].T
    aff_mat = np.exp(-1 * adj_mat / adj_mat.std())
    affMats = affMats + [aff_mat]

    sc = SpectralClustering(3, affinity='precomputed', n_init=100)
    sc.fit(aff_mat)
    for n in range(len(sc.labels_)):
        graphs[i].node[n]['group'] = str(sc.labels_[n])
    
    print(str(algos[i]), "\t", sc.labels_)
    #print(str(algos[i]), "\n", symMats[i])

In [ ]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn import datasets
from fastcluster import linkage

def seriation(Z,N,cur_index):
    '''
        input:
            - Z is a hierarchical tree (dendrogram)
            - N is the number of points given to the clustering process
            - cur_index is the position in the tree for the recursive traversal
        output:
            - order implied by the hierarchical tree Z
            
        seriation computes the order implied by a hierarchical tree (dendrogram)
    '''
    if cur_index < N:
        return [cur_index]
    else:
        left = int(Z[cur_index-N,0])
        right = int(Z[cur_index-N,1])
        return (seriation(Z,N,left) + seriation(Z,N,right))
    
def compute_serial_matrix(dist_mat,method="ward"):
    '''
        input:
            - dist_mat is a distance matrix
            - method = ["ward","single","average","complete"]
        output:
            - seriated_dist is the input dist_mat,
              but with re-ordered rows and columns
              according to the seriation, i.e. the
              order implied by the hierarchical tree
            - res_order is the order implied by
              the hierarhical tree
            - res_linkage is the hierarhical tree (dendrogram)
        
        compute_serial_matrix transforms a distance matrix into 
        a sorted distance matrix according to the order implied 
        by the hierarchical tree (dendrogram)
    '''
    N = len(dist_mat)
    flat_dist_mat = squareform(dist_mat)
    res_linkage = linkage(flat_dist_mat, method=method,preserve_input=True)
    res_order = seriation(res_linkage, N, N + N-2)
    seriated_dist = np.zeros((N,N))
    a,b = np.triu_indices(N,k=1)
    seriated_dist[a,b] = dist_mat[ [res_order[i] for i in a], [res_order[j] for j in b]]
    seriated_dist[b,a] = seriated_dist[a,b]
    
    return seriated_dist, res_order, res_linkage

In [ ]:
N = len(symMats[3])
methods = ["ward","single","average","complete"]
for method in methods:
    print("Method:\t",method)
    
    ordered_dist_mat, res_order, res_linkage = compute_serial_matrix(symMats[3]+symMats[3].T,method)
    for i in range(len(res_order)):
        print(col[res_order[i]])
    
    plt.pcolormesh(ordered_dist_mat)
    plt.xlim([0,N])
    plt.ylim([0,N])
    plt.colorbar()
    plt.show()

In [ ]:
for i in range(len(graphs)):
    G=graphs[i]
    #nx.write_gexf(G,"../output/"+str(algos[i])+".gexf")